# Advanced Search
Find entities that have a tabular_schema

In [ ]:
# Set Environment Variables
%env PURVIEW_ACCOUNT_NAME=YOUR_PURVIEW_ACCOUNT_NAME

In [ ]:
# Helper Methods
import json
def getJSON(raw_output):
    output = ''.join(raw_output)
    json_obj = json.loads(output)
    return json_obj

In [ ]:
# Get Search Results
data = !pvw search query --keywords "*" --limit 1000
data = getJSON(data)

In [ ]:
# Get Bulk Entities
import math
entity_collection = []
guid_args = ""
counter, batch, batch_size = 0, 0, 100
number_of_batches = math.ceil(len(data['value'])/batch_size)

for item in data['value']:
    counter += 1
    guid_args += f" --guid {item['id']}"
    if counter == batch_size:
        batch += 1
        print(f'Batch {batch} of {number_of_batches}')
        bulk = !pvw entity readBulk {guid_args}
        bulk = getJSON(bulk)
        
        for item in bulk['entities']:
            entity_collection.append(item)
        counter, guid_args = 0, ""

batch += 1
print(f'Batch {batch} of {number_of_batches}')
bulk = !pvw entity readBulk {guid_args}
bulk = getJSON(bulk)
for item in bulk['entities']:
    entity_collection.append(item)

In [ ]:
# Loop through relationshipAttributes for tabular_schema
for entity in entity_collection:
    for attr in entity['relationshipAttributes']:
        if attr == 'tabular_schema':
            print(f"{entity['typeName']}\t{entity['attributes']['name']}")